In [1]:
import numpy as np
from OPX_live_controller import VirtualGateSetMeasurement, QuAM
from qmucph.quam_mods.quam_mods import InOutSingleChannel_M
from quam.components.virtual_gate_set import VirtualGateSet
from quam.core import QuamRoot, quam_dataclass
from quam.components import SingleChannel, StickyChannelAddon, pulses, VirtualPulse
from dataclasses import field
from typing import List, Dict
from qm.qua import program, declare, fixed, for_, stream_processing, declare_stream, save, ramp_to_zero, align, wait
from qualang_tools.loops import from_array
import pprint
from copy import deepcopy


dividers = {'g1':1, 'g2':2, 'g3':3}
virt_rand = np.random.uniform(0,1, (3,2)) #np.ones((2,2)) - np.eye(2,2)*2 #
virtualisation_matrix = virt_rand * np.array(list(dividers.values()))[:,None]
machine = QuAM()
machine.gates = {
    "g1": SingleChannel(
        id="g1",
        opx_output=("con1", 1),
        sticky=StickyChannelAddon(duration=200, digital=False),
    ),
    "g2": SingleChannel(
        id="g2",
        opx_output=("con1", 2),
        sticky=StickyChannelAddon(duration=200, digital=False),
    ),
    "g3": SingleChannel(
        id="g3",
        opx_output=("con1", 3),
        sticky=StickyChannelAddon(duration=200, digital=False),
    ),
}

machine.VirtualGateSet1 = VirtualGateSet(
    gates=["#/gates/g1", "#/gates/g2", "#/gates/g3"], # Be careful with ordering
    virtual_gates={"v1": virtualisation_matrix[:,0],
                    "v2": virtualisation_matrix[:,1],
                    }, 
    pulse_defaults=[
        pulses.SquarePulse(amplitude=None, length=None),
        pulses.SquarePulse(amplitude=None, length=None),
        pulses.SquarePulse(amplitude=None, length=None),
    ],
)

machine.VirtualGateSet2 = VirtualGateSet(
    gates=["#/gates/g1", "#/gates/g2", "#/gates/g3"], # Be careful with ordering
    virtual_gates={"v1": virtualisation_matrix[:,0],
                    "v2": virtualisation_matrix[:,1],
                    }, 
    pulse_defaults=[
        pulses.SquarePulse(amplitude=None, length=None),
        pulses.SquarePulse(amplitude=None, length=None),
        pulses.SquarePulse(amplitude=None, length=None),
    ],
)

machine.resonator = InOutSingleChannel_M(
    id="readout",
    opx_output=("con1", 1),
    opx_input=("con1", 1),
    intermediate_frequency=192307692,
    time_of_flight=372
    )

# config = machine.generate_config()

measurer = VirtualGateSetMeasurement(QuAM= machine,
                                    resolution=51,
                                    readout_time_us= 1,
                                    readout_amplitude=0.1,
                                    dividers = dividers)


2024-06-14 15:33:29,661 - qm - INFO     - Starting session: 0f3d996d-81b8-4709-89fd-c435c474dcfa


In [2]:
virt_rand

array([[0.89676927, 0.28785859],
       [0.28683284, 0.80593909],
       [0.06277463, 0.75493533]])

In [3]:
virtualisation_matrix

array([[0.89676927, 0.28785859],
       [0.57366567, 1.61187818],
       [0.1883239 , 2.26480598]])

In [9]:
virtualisation_matrix = virt_rand #np.array([[1,0],[0,1]])

new = measurer.change_virtualisation_matrix(None, None,virtualisation_matrix)['waveforms']
# print(new)
def get_nums(waveforms_dict):
    a = []
    for gate in ['g1','g2', 'g3']:
        for pulse in ['.big_pulse.wf','.slow_pulse.wf','.small_pulse.wf']:
            if isinstance(waveforms_dict[gate+pulse], dict):
                a.append(waveforms_dict[gate+pulse]['sample'])
            else:
                a.append(waveforms_dict[gate + pulse])
    return np.array(a)
test_change = get_nums(new)
test_virtual_gate_set = get_nums(measurer.config['waveforms'])
print(test_change)
print(test_virtual_gate_set)

print(np.array(test_virtual_gate_set) - np.array(test_change))


[0.01439293 0.04483846 0.00057572 0.08059391 0.02868328 0.00322376
 0.1132403  0.0094162  0.00452961]
[0.01439293 0.04483846 0.00057572 0.08059391 0.02868328 0.00322376
 0.1132403  0.0094162  0.00452961]
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.38777878e-17  0.00000000e+00
 -8.67361738e-19]


In [5]:
virt_rand

array([[0.89676927, 0.28785859],
       [0.28683284, 0.80593909],
       [0.06277463, 0.75493533]])

In [6]:
dividers = {'g1':7.5, 'g2':9.5}
virtualisation_matrix = np.array([[1,0],[0,1]]) * np.array(list(dividers.values()))

In [7]:
virtualisation_matrix

array([[7.5, 0. ],
       [0. , 9.5]])